In [36]:
import os
import sys
import cv2
import importlib

ROOT_DIR = os.path.abspath(".")

sys.path.append(ROOT_DIR)

In [37]:
import numpy as np
import math
import mrcnn.model as modellib

import BarColor_randomcolor


import BarColor_randomcolor.ICRNConfigure
from mrcnn.config import Config



In [38]:
from BarColor_randomcolor.Dataset_generator import GenerateOneBarChart

In [39]:
ICRN_config = BarColor_randomcolor.ICRNConfigure.Config()
from BarColor_randomcolor.utils.non_local import non_local_block


In [40]:
BarColor_randomcolor.ICRNConfigure.MakeDir('testDataset')
BarColor_randomcolor.ICRNConfigure.ClearDir('testDataset')

Resetting the folder..... testDataset


In [41]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "CQA"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
maskrcnnconfig = ShapesConfig()
maskrcnnconfig.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  128
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  128
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [128 128   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [42]:
RCNNMODEL_DIR = os.path.join(ROOT_DIR, "MaskRCNNlogs/BarChart_color")

In [43]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
maskrcnn_model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=RCNNMODEL_DIR)

maskrcnn_model_path = maskrcnn_model.find_last()

# Load trained weights
print("Loading weights from ", maskrcnn_model_path)
maskrcnn_model.load_weights(maskrcnn_model_path, by_name=True)

Loading weights from  /home/mahsa.geshvadi001/projects/PaperCodes/2_Comparison/Instance-based-RN/Codes_PureExperiment_default_5_times/Task1_ourNewTasks/MaskRCNNlogs/BarChart_color/cqa20230313T1326/mask_rcnn_cqa_0060.h5
Re-starting from epoch 60


In [44]:
image_num = 1
min_num_obj = 3
max_num_obj = 6

In [71]:
def get_segmented_image(segments_bbs, image):
    
    removed_image = image.copy()
    for i in range(len(segments_bbs)):
        toremovesegment = segments_bbs[i]
    
        x1 = toremovesegment[0]
        y1  = toremovesegment[2]
        x2 = toremovesegment[1]
        y2 = toremovesegment[3]

        removed_image[x1 : y1, x2 : y2] = (1, 1, 1)
        removed_image[x1 : y1, x2 : y2] =removed_image[x1 : y1, x2 : y2] +  np.random.uniform(0, 0.05, (abs(y1-x1), abs(y2-x2),3))
        _min = 0.0  # because the image is not 0/1 black-and-white image, is a RGB image.
        _max = removed_image.max()
        removed_image -= _min
        removed_image[x1 : y1, x2 : y2] /= (_max - _min)
   
    return removed_image

In [72]:
def saveImage(image, featureVector, i):

    file_gt = open('testDataset/ground_truth.txt', 'w')
    file_pair_gt = open('testDataset/pair_ground_truth.txt','w')
    cv2.imwrite('testDataset/' + ICRN_config.chartName.format(i), image * 255)
    
    for t in range(len(featureVector)):
        file_gt.write("%.6f\t" % (featureVector[t]))
                
    for t in range(ICRN_config.max_obj_num - len(featureVector)):
        file_gt.write("0.00\t")
        file_gt.write("\n")

    for t in range(len(featureVector) - 1):
            file_pair_gt.write("{} {} {}\n".format(ICRN_config.subChartName.format(i, t),
                                                       ICRN_config.subChartName.format(i, t + 1),
                                                       featureVector[t+1] / featureVector[t]))


In [106]:
_images = np.ones((ICRN_config.max_obj_num, image_num, ICRN_config.image_height, ICRN_config.image_width, 3), dtype='float32')
_labels = []
number_of_bars = []

In [107]:
for i in range(image_num):

        image, _, featureVector = GenerateOneBarChart(
                num=np.random.randint(min_num_obj, max_num_obj + 1))
        
        saveImage(image, featureVector, i)
        featureVector = np.array(featureVector)

        results = maskrcnn_model.detect([image], verbose=1)
        r = results[0]
        arr = r['rois']
        segments_bbs = arr[arr[:,1].argsort()]

        segments = []
        for t in range(len(r['rois'])):
            segments.append(get_segmented_image([x for x in segments_bbs if ((x != segments_bbs[t]).any())], image))

            
        subImages = [np.ones(shape=(ICRN_config.image_width,ICRN_config.image_width,3)) for i in range(ICRN_config.max_obj_num)]
        for count in range(len(r['rois'])):

            if count< 6:
                subImages[count] = segments[count]
            
        for t in range(ICRN_config.max_obj_num):
            _images[t][i] = subImages[t]

        number_of_bars.append(len(r['rois']))
        
        label = np.zeros(ICRN_config.max_obj_num, dtype='float32')
        label[:len(featureVector)] = featureVector
        _labels.append(label)
        
_labels = np.array(_labels, dtype='float32')

Processing 1 images
image                    shape: (100, 100, 3)         min:    0.04317  max:    1.00000  float64
molded_images            shape: (1, 128, 128, 3)      min: -123.65312  max: -102.90102  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  128.00000  float64
anchors                  shape: (1, 4092, 4)          min:   -0.71267  max:    1.20874  float32


In [108]:
x_test = _images
x_test -= .5


In [109]:
input_test = [x_test[i] for i in range(ICRN_config.max_obj_num)]
#x_test.append(np.ones(image_num))
#input_test = [x_test[i] for i in range(ICRN_config.max_obj_num)]
#input_test.append(np.ones(input_test[0].shape[0]))

# Predict with RN Seg

In [77]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from keras.optimizers import SGD, Adam
import sklearn
from sklearn.metrics import mean_squared_error

In [78]:

def Build_RN_Network():
    # input layers.
    input_layers = []
    for i in range(ICRN_config.max_obj_num):
        input = Input(shape=(ICRN_config.image_height, ICRN_config.image_width, 3), name="input_{}".format(i))
        input_layers.append(input)

    # First extract individual features.
    individual_features = []
    for i in range(ICRN_config.max_obj_num):
        x = Conv2D(24,(5,5),strides=(3,3),activation='relu',padding='same')(input_layers[i])
        x = Conv2D(24,(5,5),strides=(3,3),activation='relu',padding='same')(x)
        x = Conv2D(24,(5,5),strides=(2,2),activation='relu',padding='same')(x)
        x = Conv2D(24,(5,5),strides=(2,2),activation='relu',padding='same')(x)

        x = Flatten()(x)

        x = Dense(24, activation='relu')(x)

        individual_features.append(x)

    # g_theta
    g = []
    for i in range(len(individual_features)):
        for j in range(len(individual_features)):
            x = keras.layers.concatenate([individual_features[i], individual_features[j]])
            x = Dense(256, activation="relu")(x)
            x = Dense(256, activation="relu")(x)
            x = Dense(256, activation="relu")(x)
            x = Dense(256, activation="relu")(x)
            g.append(x)
    f = keras.layers.average(g)

    # f_theta
    z = Dense(256, activation="relu")(f)
    z = Dense(256, activation="relu")(z)
    z = Dropout(0.5)(z)
    z = Dense(ICRN_config.max_obj_num, activation="linear")(z)

    return Model(inputs=input_layers, outputs=z)

In [100]:
m_optimizer = Adam(0.0001)

model = Build_RN_Network()
model.compile(loss='mse', optimizer=m_optimizer)

In [101]:
ICRNModelPath = os.path.join(ROOT_DIR, "results/IRN/BarColor")

In [102]:
model.load_weights(ICRNModelPath + '/Barchart_color.h5')

In [110]:
predict_Y = model.predict(x=input_test, batch_size=1)


In [111]:
for n in range(image_num):

        predict_Y[n][number_of_bars[n]: 6] = 0

In [112]:
y = _labels

MLAE = np.log2(sklearn.metrics.mean_absolute_error( predict_Y * 100, y * 100) + .125)

In [113]:
MLAE

0.18261258021868437